# Explorativ Data Analyse

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

sns.set()

In [2]:
df_data = pd.read_csv('data/ecommerce-data.zip', encoding='ISO-8859-1')

In [3]:
df_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Lad os først forstå dataset.

Hvor mange rækker er der?

In [4]:
print(f'Antal transaktioner: {len(df_data)}')

Antal transaktioner: 541909


Har vi dubletter?

In [5]:
n_duplicates = len(df_data[df_data.duplicated()])
print(f'Antallet af dubletter: {n_duplicates}')

Antallet af dubletter: 5268


Der er nogle enkelte dubletter. Disse vil ødelægge vores statistikker, så vi sletter dem.

In [6]:
df_data[df_data.duplicated()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,12/1/2010 11:45,1.25,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,12/1/2010 11:45,2.10,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,12/1/2010 11:45,2.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,12/1/2010 11:45,4.95,17908.0,United Kingdom
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,12/1/2010 11:49,2.95,17920.0,United Kingdom
...,...,...,...,...,...,...,...,...
541675,581538,22068,BLACK PIRATE TREASURE CHEST,1,12/9/2011 11:34,0.39,14446.0,United Kingdom
541689,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,12/9/2011 11:34,2.49,14446.0,United Kingdom
541692,581538,22992,REVOLVER WOODEN RULER,1,12/9/2011 11:34,1.95,14446.0,United Kingdom
541699,581538,22694,WICKER STAR,1,12/9/2011 11:34,2.10,14446.0,United Kingdom


In [7]:
n_before = len(df_data)
df_data.drop_duplicates(inplace=True)
n_after = len(df_data)
print(f'Slettet {n_before - n_after} dubletter.')

Slettet 5268 dubletter.


Hvor mange værdier manger vi?

In [8]:
df_na = pd.DataFrame(df_data.isnull().sum(), columns=['NumberOfMissingValues']).T
df_na

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
NumberOfMissingValues,0,0,1454,0,0,0,135037,0


In [9]:
n_transactions_with_no_cust_info = df_na['CustomerID'][0]
pct_transactions_no_cust = n_transactions_with_no_cust_info / len(df_data) * 100
print(f'Andel af transaktioner uden kunde-ID: {pct_transactions_no_cust:.2f}%')

Andel af transaktioner uden kunde-ID: 25.16%


**OBS!** Der er relativ mange transaktioner uden kunde-ID.
Forklaring kunne være at kunde-ID er kun sat til transaktioner fra forbrugere
mens grossister ikke har en kunde-ID. Bekræft denne hypotese.

## Split data i to DateFrames

Data er joinet. Vi splitter det op for at få et bedre overblik.

In [10]:
df_invoices = df_data[['StockCode', 'InvoiceNo']].groupby(['InvoiceNo']).count()
df_invoices = df_invoices.rename(columns={'StockCode': 'NumberOfItems'})

In [11]:
df_data['Revenue'] = df_data['Quantity'] * df_data['UnitPrice']
df_invoices['TotalRevenue'] = df_data[['Revenue', 'InvoiceNo']].groupby(['InvoiceNo']).sum()

In [12]:
# Antager at kunde, land og dato er angivet på faktura-niveau
invoice_fields = ['CustomerID', 'InvoiceDate', 'Country']
df_invoices['CustomerID'] = df_data[['CustomerID', 'InvoiceNo']].groupby(['InvoiceNo']).first()
df_invoices['Date'] = df_data[['InvoiceDate', 'InvoiceNo']].groupby(['InvoiceNo']).first()
df_invoices['Country'] = df_data[['Country', 'InvoiceNo']].groupby(['InvoiceNo']).first()

In [13]:
df_invoices.reset_index(inplace=True)

In [14]:
df_invoices.head()

,InvoiceNo,NumberOfItems,TotalRevenue,CustomerID,Date,Country
0,536365,7,139.12,17850.0,12/1/2010 8:26,United Kingdom
1,536366,2,22.20,17850.0,12/1/2010 8:28,United Kingdom
2,536367,12,278.73,13047.0,12/1/2010 8:34,United Kingdom
3,536368,4,70.05,13047.0,12/1/2010 8:34,United Kingdom
4,536369,1,17.85,13047.0,12/1/2010 8:35,United Kingdom


In [15]:
df_invoice_lines = df_data[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice', 'Revenue']]

In [16]:
df_invoice_lines.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,15.30
1,536365,71053,WHITE METAL LANTERN,6,3.39,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34


## Faktura nummer (InvoiceNo)

Lad os først undersøge de enkelte kolonner.

In [17]:
df_invoices.InvoiceNo

0         536365
1         536366
2         536367
3         536368
4         536369
          ...   
25895    C581484
25896    C581490
25897    C581499
25898    C581568
25899    C581569
Name: InvoiceNo, Length: 25900, dtype: object

Nogle fakturanumre har et C foran. Hvorfor?

In [18]:
df_invoices['AllDigits'] = df_invoices.InvoiceNo.map(lambda v: v.isdigit())

In [19]:
df_invoices

,InvoiceNo,NumberOfItems,TotalRevenue,CustomerID,Date,Country,AllDigits
0,536365,7,139.12,17850.0,12/1/2010 8:26,United Kingdom,True
1,536366,2,22.20,17850.0,12/1/2010 8:28,United Kingdom,True
2,536367,12,278.73,13047.0,12/1/2010 8:34,United Kingdom,True
3,536368,4,70.05,13047.0,12/1/2010 8:34,United Kingdom,True
4,536369,1,17.85,13047.0,12/1/2010 8:35,United Kingdom,True
...,...,...,...,...,...,...,...
25895,C581484,1,-168469.60,16446.0,12/9/2011 9:27,United Kingdom,False
25896,C581490,2,-32.53,14397.0,12/9/2011 9:57,United Kingdom,False
25897,C581499,1,-224.69,15498.0,12/9/2011 10:28,United Kingdom,False
25898,C581568,1,-54.75,15311.0,12/9/2011 11:57,United Kingdom,False


Interessant. Indtægt er negativt. Måske indikerer 'C'-et en kreditnota?
Kan vi bekræfte den antagelse?

In [20]:
df_invoices[df_invoices.AllDigits == False][['TotalRevenue']].describe().T

,count,mean,std,min,25%,50%,75%,max
TotalRevenue,3839.0,-235.749359,3209.245653,-168469.6,-41.375,-16.85,-7.95,11062.06


Det er ikke alle steder hvor det er negativt. Lad os finde faktura med positiv indtjening.

In [21]:
df_invoices[(df_invoices.AllDigits == False) & (df_invoices.TotalRevenue >= 0)]

,InvoiceNo,NumberOfItems,TotalRevenue,CustomerID,Date,Country,AllDigits
22061,A563185,1,11062.06,NaN,8/12/2011 14:50,United Kingdom,False


Der er faktura som starter med A. Hvad mon det betyder?

In [22]:
df_invoice_lines[df_invoice_lines.InvoiceNo.str.startswith('A')]

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,Revenue
299982,A563185,B,Adjust bad debt,1,11062.06,11062.06
299983,A563186,B,Adjust bad debt,1,-11062.06,-11062.06
299984,A563187,B,Adjust bad debt,1,-11062.06,-11062.06


Det lader til at der er tale om afskrivningsposter.

In [23]:
filter_A = df_invoices.InvoiceNo.str.startswith('A')
filter_C = df_invoices.InvoiceNo.str.startswith('C')
n_invoices_A = len(df_invoices[filter_A])
n_invoices_C = len(df_invoices[filter_C])
n_invoices_rest = len(df_invoices[~filter_A & ~filter_C])
n_invoices_total = len(df_invoices)

print(f'Antal faktura: {n_invoices_total}')
print(f'Antal faktura med A-nummer: {n_invoices_A}')
print(f'Antal faktura med C-nummer: {n_invoices_C}')
print(f'Antal faktura uden prefix: {n_invoices_rest}')

Antal faktura: 25900
Antal faktura med A-nummer: 3
Antal faktura med C-nummer: 3836
Antal faktura uden prefix: 22061


Umiddelbart ser det ud som om at der er tre typer af faktura:
- Salgsfaktura (intet bogstav i fakturanummer)
- Kreditnota (fakturanummer starter med et C)
- Afskrivning (fakturanummer starter med et A)

In [24]:
def get_type(invoice_no):
    if invoice_no[0] == 'A':
        return 'Write-Off'
    if invoice_no[0] == 'C':
        return 'Credit Note'
    return 'Standard'

df_invoices['Type'] = df_invoices.InvoiceNo.map(get_type)

In [25]:
df_invoices.head()

,InvoiceNo,NumberOfItems,TotalRevenue,CustomerID,Date,Country,AllDigits,Type
0,536365,7,139.12,17850.0,12/1/2010 8:26,United Kingdom,True,Standard
1,536366,2,22.20,17850.0,12/1/2010 8:28,United Kingdom,True,Standard
2,536367,12,278.73,13047.0,12/1/2010 8:34,United Kingdom,True,Standard
3,536368,4,70.05,13047.0,12/1/2010 8:34,United Kingdom,True,Standard
4,536369,1,17.85,13047.0,12/1/2010 8:35,United Kingdom,True,Standard


## Omsætning per faktura

In [26]:
df_invoices[df_invoices.TotalRevenue == 0]

,InvoiceNo,NumberOfItems,TotalRevenue,CustomerID,Date,Country,AllDigits,Type
43,536414,1,0.0,NaN,12/1/2010 11:52,United Kingdom,True,Standard
85,536545,1,0.0,NaN,12/1/2010 14:32,United Kingdom,True,Standard
86,536546,1,0.0,NaN,12/1/2010 14:33,United Kingdom,True,Standard
87,536547,1,0.0,NaN,12/1/2010 14:33,United Kingdom,True,Standard
88,536549,1,0.0,NaN,12/1/2010 14:34,United Kingdom,True,Standard
...,...,...,...,...,...,...,...,...
21904,581226,1,0.0,NaN,12/8/2011 9:56,United Kingdom,True,Standard
21910,581234,1,0.0,NaN,12/8/2011 10:33,United Kingdom,True,Standard
21962,581406,2,0.0,NaN,12/8/2011 13:58,United Kingdom,True,Standard
21964,581408,1,0.0,NaN,12/8/2011 14:06,United Kingdom,True,Standard
